In [1]:
# Import modules
from tika import parser # PDF file parser
import pandas as pd
import glob # Unix style pathname pattern expansion
from Bio import SeqIO
from geopy.geocoders import Nominatim # Module to get coordinates
import psycopg2
from datetime import date

In [2]:
# Function to replace spaces to underscores in strings
def replace_spaces(string):
    return '_'.join(string.split(' '))

def remove_spaces(string):
    return '/'.join(string.split(' / '))


# Function to read the GISAID metadata PDF file and filter out unnecessary lines/fields
def read_pdf(path):
    pdf = parser.from_file(path)
    raw_content = pdf['content'].lstrip('\n')
    content_lines = raw_content.split('\n')
    filtered_content = list(filter(lambda x: x != '' and x != 'Virus detail' and x != 'Sample information' 
                          and x != 'Institute information' and x != 'Submitter information' 
                                   and (not x.lower().startswith('updated')) 
                                   and (not x.lower().startswith('note')) 
                                   and (not x.lower().startswith('please note'))
                                   and (not x.lower().startswith('apartado')), content_lines))
    return filtered_content[1:]

In [3]:
# Process text
def fix_content(content):
    
    authors = ''
    modified_content = []
    
    for i in range(len(content)):
        if content[i] == 'Sample ID given by the sample' and content[i+1] == 'provider:':
            content[i] = content[i] + ' ' + content[i+1]
            content.pop(i+1)
        if content[i] == 'Sample ID given by the' and content[i+1] == 'submitting laboratory:':
            content[i] = content[i] + ' ' + content[i+1]
            content.pop(i+1)

        if content[i] == 'Additional location' and content[i+1] == 'information:':
            content[i] = content[i] + ' ' + content[i+1]
            content.pop(i+1)
            
        
        if content[i].startswith('Originating lab'):
            
            while not content[i+1].startswith('Address'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i+1)
        
        if content[i].startswith('Submitting lab'):
            
            while not content[i+1].startswith('Address'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i+1)
                
        
        if content[i].startswith('Sample ID given by the sample provider'):
            if not content[i+1].startswith('Submitting lab'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i+1)
                
        
        if content[i].startswith('Additional location'):
            if not content[i + 1].startswith('information') and not content[i+1].startswith('Gender'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i+1)
                
        
        if content[i].startswith('Sample ID given by the submitting laboratory'):
            if not content[i+1].startswith('Authors'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i+1)
                
                
        if content[i].startswith('Comment'):
            
            while not content[i + 1].startswith('Originating'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i+1)                                             

        if content[i].startswith('Authors'):
            authors = content[i]
            
            while not content[i + 1].startswith('Submitter'):
                authors = authors + ' ' + content[i + 1]
                content.pop(i + 1)
            content[i] = authors
             

        if (content[i].startswith('Address:')) and (not content[i+1].startswith('Sample')):
            if not content[i + 1].startswith('Important note'):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i + 1)
                
                
            while (not content[i + 1].startswith('Important note') and (content[i - 1].startswith('Submission'))):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i + 1)
            
            while (not content[i + 1].startswith('Sample') and (content[i-1].startswith('Originating'))):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i + 1)
                
            while (not content[i + 1].startswith('Sample') and (content[i-1].startswith('Submitting'))):
                content[i] = content[i] + ' ' + content[i + 1]
                content.pop(i + 1)
                        
        if content[i].startswith('Important note'):
            break
        modified_content.append(content[i].split(':'))
    
    return modified_content

In [4]:
# Function to create a pandas dataframe 
def create_dataframe(content):
    data_dict = {}
    for i in range(len(content)):
        
        if content[i][0] == 'Location':
            content[i][0] = replace_spaces(content[i][0])
            content[i][1] = remove_spaces(content[i][1])
            content[i][1] = replace_spaces(content[i][1])
            content[i][1] = content[i][1].lstrip('_')
        else:
            content[i][0] = replace_spaces(content[i][0])
            content[i][1] = replace_spaces(content[i][1]).lstrip('_')

        if content[i][0] == 'Address':
            if content[i-1][0] == 'Originating_lab':
                content[i][0] = 'Originating_lab_address'
            if content[i-1][0] == 'Submitting_lab':
                content[i][0] = 'Submitting_lab_address'
            if content[i-1][0] == 'Submission_Date':
                content[i][0] = 'Submitter_address'

        if content[i][1]:
            data_dict[content[i][0].lower()] = content[i][1]
        else:
            data_dict[content[i][0].lower()] = 'NA'
    
    df = pd.DataFrame([data_dict], columns=data_dict.keys())
    return df

In [5]:
# Create a list of dataframes
df_list = []
for filename in glob.glob('EPI*.pdf'):
    print(filename)
    pdf = read_pdf(filename)
    content = fix_content(pdf)
    df = create_dataframe(content)
    df_list.append(df)

EPI_ISL_402119.pdf
EPI_ISL_402120.pdf
EPI_ISL_402121.pdf
EPI_ISL_402123.pdf
EPI_ISL_402124.pdf
EPI_ISL_402125.pdf
EPI_ISL_402126.pdf
EPI_ISL_402127.pdf
EPI_ISL_402128.pdf
EPI_ISL_402129.pdf
EPI_ISL_402130.pdf
EPI_ISL_402131.pdf
EPI_ISL_402132.pdf
EPI_ISL_403928.pdf
EPI_ISL_403929.pdf
EPI_ISL_403930.pdf
EPI_ISL_403931.pdf
EPI_ISL_403932.pdf
EPI_ISL_403933.pdf
EPI_ISL_403934.pdf
EPI_ISL_403935.pdf
EPI_ISL_403936.pdf
EPI_ISL_403937.pdf
EPI_ISL_403962.pdf
EPI_ISL_403963.pdf
EPI_ISL_404227.pdf
EPI_ISL_404228.pdf
EPI_ISL_404253.pdf
EPI_ISL_404895.pdf
EPI_ISL_405839.pdf
EPI_ISL_406030.pdf
EPI_ISL_406031.pdf
EPI_ISL_406034.pdf
EPI_ISL_406036.pdf
EPI_ISL_406223.pdf
EPI_ISL_406531.pdf
EPI_ISL_406533.pdf
EPI_ISL_406534.pdf
EPI_ISL_406535.pdf
EPI_ISL_406536.pdf
EPI_ISL_406538.pdf
EPI_ISL_406592.pdf
EPI_ISL_406593.pdf
EPI_ISL_406594.pdf
EPI_ISL_406595.pdf
EPI_ISL_406596.pdf
EPI_ISL_406597.pdf
EPI_ISL_406716.pdf
EPI_ISL_406717.pdf
EPI_ISL_406798.pdf
EPI_ISL_406799.pdf
EPI_ISL_406800.pdf
EPI_ISL_4068

EPI_ISL_413889.pdf
EPI_ISL_413890.pdf
EPI_ISL_413891.pdf
EPI_ISL_413892.pdf
EPI_ISL_413893.pdf
EPI_ISL_413894.pdf
EPI_ISL_413895.pdf
EPI_ISL_413896.pdf
EPI_ISL_413897.pdf
EPI_ISL_413898.pdf
EPI_ISL_413899.pdf
EPI_ISL_413900.pdf
EPI_ISL_413901.pdf
EPI_ISL_413902.pdf
EPI_ISL_413904.pdf
EPI_ISL_413905.pdf
EPI_ISL_413906.pdf
EPI_ISL_413922.pdf
EPI_ISL_413924.pdf
EPI_ISL_413925.pdf
EPI_ISL_413926.pdf
EPI_ISL_413928.pdf
EPI_ISL_413931.pdf
EPI_ISL_413949.pdf
EPI_ISL_413955.pdf
EPI_ISL_413996.pdf
EPI_ISL_413997.pdf
EPI_ISL_413999.pdf
EPI_ISL_414005.pdf
EPI_ISL_414006.pdf
EPI_ISL_414007.pdf
EPI_ISL_414008.pdf
EPI_ISL_414009.pdf
EPI_ISL_414010.pdf
EPI_ISL_414011.pdf
EPI_ISL_414012.pdf
EPI_ISL_414013.pdf
EPI_ISL_414014.pdf
EPI_ISL_414015.pdf
EPI_ISL_414016.pdf
EPI_ISL_414017.pdf
EPI_ISL_414019.pdf
EPI_ISL_414020.pdf
EPI_ISL_414021.pdf
EPI_ISL_414022.pdf
EPI_ISL_414023.pdf
EPI_ISL_414024.pdf
EPI_ISL_414025.pdf
EPI_ISL_414026.pdf
EPI_ISL_414027.pdf
EPI_ISL_414040.pdf
EPI_ISL_414041.pdf
EPI_ISL_4140

EPI_ISL_415589.pdf
EPI_ISL_415590.pdf
EPI_ISL_415591.pdf
EPI_ISL_415592.pdf
EPI_ISL_415593.pdf
EPI_ISL_415594.pdf
EPI_ISL_415595.pdf
EPI_ISL_415596.pdf
EPI_ISL_415597.pdf
EPI_ISL_415598.pdf
EPI_ISL_415599.pdf
EPI_ISL_415600.pdf
EPI_ISL_415601.pdf
EPI_ISL_415602.pdf
EPI_ISL_415603.pdf
EPI_ISL_415604.pdf
EPI_ISL_415605.pdf
EPI_ISL_415606.pdf
EPI_ISL_415607.pdf
EPI_ISL_415608.pdf
EPI_ISL_415609.pdf
EPI_ISL_415610.pdf
EPI_ISL_415611.pdf
EPI_ISL_415612.pdf
EPI_ISL_415613.pdf
EPI_ISL_415614.pdf
EPI_ISL_415615.pdf
EPI_ISL_415616.pdf
EPI_ISL_415617.pdf
EPI_ISL_415618.pdf
EPI_ISL_415619.pdf
EPI_ISL_415620.pdf
EPI_ISL_415621.pdf
EPI_ISL_415622.pdf
EPI_ISL_415623.pdf
EPI_ISL_415624.pdf
EPI_ISL_415625.pdf
EPI_ISL_415626.pdf
EPI_ISL_415627.pdf
EPI_ISL_415628.pdf
EPI_ISL_415629.pdf
EPI_ISL_415630.pdf
EPI_ISL_415631.pdf
EPI_ISL_415632.pdf
EPI_ISL_415633.pdf
EPI_ISL_415634.pdf
EPI_ISL_415635.pdf
EPI_ISL_415636.pdf
EPI_ISL_415637.pdf
EPI_ISL_415638.pdf
EPI_ISL_415639.pdf
EPI_ISL_415640.pdf
EPI_ISL_4156

In [6]:
# Concatenate dataframes into 1 dataframe
data = pd.concat(df_list, sort=False)

In [7]:
# Get unique locations
locations = (data['location'].unique())
# Create a coordinate dictionary and initiate geolocator object
coordinates = {}
geolocator = Nominatim(user_agent='covid-locations')

def remove_underscore(string):
    return ' '.join(string.split('_'))

# Get coordinates for each location
def get_coords(locations):
    for location in locations:
        precise_location = ''
        if location == 'North_America/USA/Washington':
            precise_location = 'Washington, Seattle'
        else:
            if location.startswith('North_America'):
                precise_location = remove_underscore(', '.join(location.split('/')[-2:]))
            else:
                precise_location = ' '.join(location.split('/')[-1].split('_'))
        loc_coordinates = geolocator.geocode(precise_location)
        print(location)
        coordinates[location] = [loc_coordinates.latitude, loc_coordinates.longitude]
        


# Retrieve data in chucnk to avoid errors if there are too many requests
# First 50 locations
get_coords(locations[:50])

Asia/China/Hubei/Wuhan
Asia/China
Asia/Japan/Kanagawa
Asia/China/Yunnan/Pu'er
Asia/China/Guandong/Shenzhen
Asia/China/Guangdong/Shenzhen
Asia/China/Guangdong/Zhuhai
Asia/Thailand/Nonthaburi
Asia/China/Zhejiang
North_America/USA/Illinois/Chicago
North_America/USA/Washington/Snohomish_County
Asia/Taiwan/Kaohsiung
North_America/USA/California/Los_Angeles
North_America/USA/California/Orange_County
North_America/USA/Arizona/Phoenix
Asia/China/Guangdong/Guangzhou
Asia/China/Guangdong/Foshan
Asia/China/Guangdong
Europe/France/Ile-de-France/Paris
Asia/China/Wuhan
Oceania/Australia/Victoria/Clayton
Europe/Germany/Bavaria/Munich
Europe/Italy/Rome
Asia/China/Zhejiang/Hangzhou
Asia/Singapore
Europe/England
Europe/Finland/Lapland
Asia/Japan/Aichi
Asia/South_Korea/Gyeonggi-do
North_America/USA/Washington
Oceania/Australia/New_South_Wales/Sydney
Oceania/Australia/Queensland/Gold_Coast
Europe/Belgium/Leuven
North_America/USA/California
Asia/China/Chongqinq/Yongchuan
Asia/China/Chongqing/Zhongxian
Asia

In [8]:
# 50 - 99
get_coords(locations[50:100])

Asia/Hong_Kong
North_America/USA/Massachusetts
North_America/USA/Illinois
Asia/Nepal/Kathmandu
Asia/Philippines
Europe/France/Rhone-Alpes/Contamines
Asia/Malaysia
Asia/Japan/Nara
Asia/Japan/Osaka
Asia/China/Guangxi
Asia/China/Fujian
Asia/Cambodia/Sihanoukville
Asia/Taiwan/Taoyuan
Asia/South_Korea
Asia/China/Jiangsu
Europe/Sweden
North_America/USA/Texas
Asia/China/HuaShang
Asia/China/Anhui/Hefei
Asia/China/Shenzhen
Asia/China/Shandong/Linyi_county
Asia/China/Shandong/Linyi
Asia/China/Hubei/Jingzhou
North_America/USA/California/Solano
Asia/South_Korea_/Seoul
Asia/South_Korea/_Seoul
Asia/South_Korea/Seoul
Asia/South_Korea/Chungcheongnam-do
Europe/Germany/Baden-Wuerttemberg
South_America/Brazil/Sao_Paulo/Sao_Paulo
North_America/Canada/British_Columbia
Asia/China/Guangzhou
Asia/Japan
Europe/Finland/Helsinki
North_America/Mexico/Mexico_City
Europe/Italy/Lombardy
Asia/China/Hubei/Tianmen
North_America/Canada/Ontario
South_America/Brazil/Sao_Paulo/_Sao_Paulo
Europe/Switzerland/Zurich
Asia/Indo

In [9]:
# 100 - 150
get_coords(locations[100:150])

Asia/Iran/Tehran
Europe/United_Kingdom/Wales
North_America/USA/California/Sonoma_County
North_America/USA/California/Solano_County
Europe/Netherlands/Andel
Europe/Netherlands/Berlicum
Europe/Netherlands/Blaricum
Europe/Netherlands/Coevorden
Europe/Netherlands/Dalen
Europe/Netherlands/Delft
Europe/Netherlands/Diemen
Europe/Netherlands/Eindhoven
Europe/Netherlands/Haarlem
Europe/Netherlands/Hardinxveld_Giessendam
Europe/Netherlands/Helmond
Europe/Netherlands/Houten
Europe/Netherlands/Loon_op_zand
Europe/Netherlands/Naarden
Europe/Netherlands/Nieuwendijk
Europe/Netherlands/Nootdorp
Europe/Netherlands/Oisterwijk
Europe/Netherlands/Oss
Europe/Netherlands/Rotterdam
Europe/Netherlands/Tilburg
Europe/Netherlands/Utrecht
Europe/Netherlands/Zeewolde
Europe/Luxemburg
Oceania/Australia/NSW/Sydney
North_America/USA
Europe/Portugal
Asia/China_/Guangdong
Asia/Iran/_Qom
North_America/USA/California/San_Francisco
Europe/Switzerland/Tessin
Europe/Switzerland/Geneva
Europe/Switzerland/Argovie
Europe/Unit

In [10]:
# 150 - N
get_coords(locations[150:])

North_America/USA/New_York
Europe/Czech_Republic/Usti_nad_Labem
Europe/Ireland/Cork
Europe/Spain/Comunitat_Valenciana/Valencia
Europe/United_Kingdom/England/Yorkshire/Sheffield
Europe/Germany/North_Rhine_Westphalia/Duesseldorf
Asia/China/_Shanghai
Europe/Italy
Asia/India
Europe/Germany/Munich
Asia/Iran/Tehran/Nazi_Abad
South_America/Chile/Talca
South_America/Chile/Santiago
Europe/Ireland/Dublin
Europe/Ireland/Limerick
North_America/USA/Minnesota
North_America/USA/Washington/Kirkland
North_America/USA/Washington/Tacoma
Europe/Spain/Comunitat_Valenciana_/Valencia
Europe/Italy/Cagliari
Europe/France/Grand-Est/Strasbourg
Europe/France/Normandie/Rouen
North_America/USA/Washington/Seattle
Europe/France/Pays_de_la_Loire/Nantes
Europe/France/Hauts_de_France/Crépy_en_Valois
Europe/France/Hauts_de_France/Compiègne
Europe/France/Grand-Est/Reims
Europe/France/Ile-de-France/Pontoise
North_America/USA/New_York/Nassau_County
Europe/Finland
Africa/Congo/Kinshasa
North_America/USA/California/San_Diego_

In [11]:
# Add coordinate data to the dataframe
for location in coordinates:
    data.loc[data.location == location, 'latitude'] = coordinates[location][0]
    data.loc[data.location == location, 'longitude'] = coordinates[location][1]

In [12]:
# Add country and city/state/province data

countries = []
cities_states_provinces = []
country_and_city_data = {}
for location in locations:
    loc = location.split('/')
    if len(loc) == 2:
        country = loc[1].lstrip('_').rstrip('_')
        loc_data = {'country': country, 'city/state/province': country}
        country_and_city_data[location] = loc_data
        countries.append(country)
    else:
        country = loc[1].lstrip('_').rstrip('_')
        city_state_province = loc[-1].lstrip('_').rstrip('_')
        countries.append(country)
        cities_states_provinces.append(city_state_province)
        loc_data = {'country': country, 'city/state/province': city_state_province}
        country_and_city_data[location] = loc_data

        
unique_countries = set(countries)
unique_cities_states_provinces = set(cities_states_provinces)

# Country ids

country_ids = {}
i = 1
for country in unique_countries:
    country_ids[country] = i
    i += 1
    
# City/State/Province ids
city_state_province_ids = {}
i = 1
for location in unique_cities_states_provinces:
    city_state_province_ids[location] = i
    i += 1
    

# Add country and city columns
for location in country_and_city_data:
    country = country_and_city_data[location]['country']
    city = country_and_city_data[location]['city/state/province']
    data.loc[data.location == location, 'country'] = country
    data.loc[data.location == location, 'city/state/province'] = city

In [13]:
# Location info
conn_string = "host='borreliabase.org' dbname='bb3-dev' user='' password=''"
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
geo_query = 'SELECT * FROM CV_GEO ORDER BY geo_id;'
country_query = 'SELECT * FROM COUNTRY ORDER BY country_id;'

def get_table(query):
    cursor.execute(query)
    records = cursor.fetchall()
    table_description = cursor.description
    
    # Column names
    cols = []
    for column in table_description:
        cols.append(column.name)
        
    # Table data
    data = []
    for row in records:
        data.append(list(row))
    df = pd.DataFrame(data, columns = cols)
    return df


geo_table = get_table(geo_query)
country_table = get_table(country_query)
geo_table['geo_name'] = geo_table['geo_name'].apply(lambda x: ':'.join([item.replace(' ', '_') for item in x.split(':')]))

In [14]:
cursor.execute(country_query)
records = cursor.fetchall()
country_id_dict = {}
for country in records:
    country_name = replace_spaces(country[1])
    if country_name == 'German':
        country_name = 'Germany'
    country_id_dict[country_name] = country[0]

In [15]:
for country in country_id_dict:
    country_name = replace_spaces(country)
    data.loc[data.country == country_name, 'country_id'] = country_id_dict[country_name]

In [16]:
# Create new location table
current_locations = [replace_spaces(loc) for loc in geo_table.geo_name.values]
new_locations = []
for loc in locations:
    temp = [item.rstrip('_').lstrip('_') for item in loc.split('/')]
    area = temp[0]
    area_id = 0
    
    if area == 'North_America':
        area_id = 1
    elif area == 'South_America':
        area_id = 2
    elif area == 'Europe':
        area_id = 3
    elif area == 'Asia':
        area_id = 4
    elif area == 'Africa':
        area_id = 5
    elif area == 'Oceania':
        area_id = 6
    elif area == 'Central_America':
        area_id = 7
    
    geo_name = ':'.join(temp[1:])
    if geo_name == 'China:Wuhan':
        geo_name = 'China:Hubei:Wuhan'
    if geo_name == 'China:Guandong:Shenzhen':
        geo_name = 'China:Guangdong:Shenzhen'
    if geo_name == 'USA:California:Los_Angeles':
        geo_name = 'USA:California:LA'
    if geo_name == 'China:Chongqinq:Yongchuan':
        geo_name = 'China:Chongqing:Yongchuan'
    if geo_name == 'Hong_Kong':
        geo_name = 'China:Hong_Kong'
    if geo_name == 'England':
        geo_name = 'United_Kingdom:England'
    if geo_name not in current_locations:
        new_locations.append([geo_name, area_id])

In [17]:
cols = ['geo_name', 'country_name', 'latitude', 'longitude', 'country_id', 'area_id', 'geo_id']
new_location_data = []
current_geo_id = max(geo_table.loc[geo_table.geo_id < 500, 'geo_id'].values) + 1

current_na_id = max(country_table.loc[(country_table.country_id < 200) 
                                      & (country_table.country_id > 100), 'country_id'].values) + 1
current_sa_id = max(country_table.loc[(country_table.country_id < 300) 
                                      & (country_table.country_id > 200), 'country_id'].values) + 1
current_eu_id = max(country_table.loc[(country_table.country_id < 400) 
                                      & (country_table.country_id > 300), 'country_id'].values) + 1
current_as_id = max(country_table.loc[(country_table.country_id < 500) 
                                      & (country_table.country_id > 400), 'country_id'].values) + 1
current_af_id = max(country_table.loc[(country_table.country_id < 600) 
                                      & (country_table.country_id > 500), 'country_id'].values) + 1
current_oc_id = max(country_table.loc[(country_table.country_id < 700) 
                                      & (country_table.country_id > 600), 'country_id'].values) + 1
# current_ca_id = max(country_table.loc[(country_table.country_id < 800) 
#                                       & (country_table.country_id > 700), 'country_id'].values) + 1
current_ca_id = 701

for loc in new_locations:
    
    temp = loc[0].split(':')
    country_name = temp[0]
    area_id = loc[1]


       
    precise_location = ''
    if len(temp) > 1:
        precise_location = ', '.join(temp[1:])
    else:
        precise_location = temp[0]
    geo_info = geolocator.geocode(remove_underscore(precise_location), language='en')
    
    country_id = 0
    if country_name in country_id_dict.keys():
        country_id = country_id_dict[country_name]
    else:
        if area_id == 1:
            country_id = current_na_id
            current_na_id += 1
        elif area_id == 2:
            country_id = current_sa_id
            current_sa_id += 1
        elif area_id == 3:
            country_id = current_eu_id
            current_eu_id += 1
        elif area_id == 4:
            country_id = current_as_id
            current_as_id += 1
        elif area_id == 5:
            country_id = current_af_id
            current_af_id += 1
        elif area_id == 6:
            country_id = current_oc_id
            current_oc_id += 1
        elif area_id == 7:
            country_id = current_ca_id
            current_ca_id += 1
            
        
        
    geo_name = loc[0]
    geo_id = current_geo_id
    current_geo_id += 1
        
    new_location_data.append(
        [
            geo_name, 
            country_name, 
            geo_info.latitude, 
            geo_info.longitude, 
            country_id, 
            area_id, 
            geo_id
        ])

new_location_df = pd.DataFrame(new_location_data, columns=cols)
new_location_df = new_location_df.sort_values('country_id')

In [18]:
geo_dict = {}
for index, row in geo_table.iterrows():
    geo_name = row.geo_name
    geo_dict[geo_name] = {
        'geo_id': row.geo_id,
        'area_id': row.area_id,
        'country_id': row.country_id
    }
    
new_geo_dict = {}
for index, row in new_location_df.iterrows():
    geo_name = row.geo_name
    new_geo_dict[geo_name] = {
        'area_id': row.area_id,
        'geo_id': row.geo_id,
        'country_id': row.country_id
    }


    
    
for location in locations:
    geo_name = ':'.join([item.rstrip('_').lstrip('_') for item in location.split('/')[1:]])
    if geo_name == 'China:Wuhan':
        geo_name = 'China:Hubei:Wuhan'
    if geo_name == 'China:Guandong:Shenzhen':
        geo_name = 'China:Guangdong:Shenzhen'
    if geo_name == 'USA:California:Los_Angeles':
        geo_name = 'USA:California:LA'
    if geo_name == 'China:Chongqinq:Yongchuan':
        geo_name = 'China:Chongqing:Yongchuan'
    if geo_name == 'Hong_Kong':
        geo_name = 'China:Hong_Kong'
    if geo_name == 'England':
        geo_name = 'United_Kingdom:England'
        
        
    data.loc[data.location == location, 'geo_name'] = geo_name
    if geo_name in geo_dict.keys(): 
        data.loc[data.location == location, 'geo_id'] = geo_dict[geo_name]['geo_id']
        data.loc[data.location == location, 'area_id'] =  geo_dict[geo_name]['area_id']
        data.loc[data.location == location, 'country_id'] = geo_dict[geo_name]['country_id']
       
    if geo_name in new_geo_dict.keys():
        data.loc[data.location == location, 'geo_id'] = new_geo_dict[geo_name]['geo_id']
        data.loc[data.location == location, 'area_id'] = new_geo_dict[geo_name]['area_id']
        data.loc[data.location == location, 'country_id'] = new_geo_dict[geo_name]['country_id']  

In [19]:
data

,virus_name,accession_id,type,passage_details/history,collection_date,location,host,additional_location_information,gender,patient_age,...,submitter_address,comment,latitude,longitude,country,city/state/province,country_id,geo_name,geo_id,area_id
0,BetaCoV/Wuhan/IVDC-HB-01/2019,EPI_ISL_402119,betacoronavirus,"Virus_Isolate,_Passage_1",2019-12-30,Asia/China/Hubei/Wuhan,Human,NA,Female,49,...,National_Institute_for_Viral_Disease_Control_a...,NaN,30.595105,114.299935,China,Wuhan,401.0,China:Hubei:Wuhan,71.0,4.0
0,BetaCoV/Wuhan/IVDC-HB-04/2020,EPI_ISL_402120,betacoronavirus,Original,2020-01-01,Asia/China/Hubei/Wuhan,Human,NA,Male,61,...,National_Institute_for_Viral_Disease_Control_a...,NaN,30.595105,114.299935,China,Wuhan,401.0,China:Hubei:Wuhan,71.0,4.0
0,BetaCoV/Wuhan/IVDC-HB-05/2019,EPI_ISL_402121,betacoronavirus,Original,2019-12-30,Asia/China/Hubei/Wuhan,Human,NA,Male,32,...,National_Institute_for_Viral_Disease_Control_a...,NaN,30.595105,114.299935,China,Wuhan,401.0,China:Hubei:Wuhan,71.0,4.0
0,BetaCoV/Wuhan/IPBCAMS-WH-01/2019,EPI_ISL_402123,betacoronavirus,Original,2019-12-24,Asia/China/Hubei/Wuhan,Human,NA,Male,65,...,"No._9_Dong_Dan_San_Tiao,_Dong_Cheng_District,_...",NaN,30.595105,114.299935,China,Wuhan,401.0,China:Hubei:Wuhan,71.0,4.0
0,BetaCoV/Wuhan/WIV04/2019,EPI_ISL_402124,betacoronavirus,Original,2019-12-30,Asia/China/Hubei/Wuhan,Human,NA,Female,49,...,"44_Xiao_Hong_Shan,_Wuhan,_Hubei_430071",NaN,30.595105,114.299935,China,Wuhan,401.0,China:Hubei:Wuhan,71.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,hCoV-19/Switzerland/GE062072020,EPI_ISL_415706,betacoronavirus,Original,2020-03-06,Europe/Switzerland,Human,NA,unknwon,32,...,NA,NA,46.798562,8.231974,Switzerland,Switzerland,307.0,Switzerland,17.0,3.0
0,hCoV-19/Switzerland/GE6679/2020,EPI_ISL_415707,betacoronavirus,Original,2020-03-08,Europe/Switzerland,Human,NA,unknown,85,...,NA,NA,46.798562,8.231974,Switzerland,Switzerland,307.0,Switzerland,17.0,3.0
0,hCoV-19/Switzerland/GE4984/2020,EPI_ISL_415708,betacoronavirus,Original,2020-03-07,Europe/Switzerland,Human,NA,unknown,39,...,NA,NA,46.798562,8.231974,Switzerland,Switzerland,307.0,Switzerland,17.0,3.0
0,hCoV-19/Hangzhou/ZJU-01/2020,EPI_ISL_415709,betacoronavirus,Vero,2020-01-25,Asia/China/Hangzhou,Human,NA,Male,36,...,NA,NA,30.248963,120.205234,China,Hangzhou,401.0,China:Hangzhou,81.0,4.0


In [20]:
current_date = date.today()
file_date = str(current_date.month) + '-' + str(current_date.day)

# Write to file
data.to_excel('gisaid-covid19-' + file_date + '.xlsx', index=False)
data.to_csv('gisaid-covid19-' + file_date + '.tsv', index=False, sep='\t')
new_location_df.to_csv('new-geo-data-' + file_date + '.tsv', sep='\t')

## FASTA files

In [21]:
records = []
for fasta in glob.glob('EPI*.fasta'):
    fh = open(fasta)
    sequence = SeqIO.read(fh, 'fasta')
    fh.close()
    records.append(sequence)

In [22]:
SeqIO.write(records, 'covid-' + file_date + '.fasta', 'fasta')

958